In [25]:
import pandas as pd
from sqlalchemy import create_engine
import regex as re
import time
from datetime import datetime
import os

# prod
user = 'read_only'
pwd = 'WHC63fKyjudfSz2U'
host = 'ezcapc-webdb-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
db_name = 'CAPC_APIGATEWAY'
env='production'


# # stag
# user = 'read_only'
# pwd = '7TYtBma28775c4Ua'
# host = 'ezcapc-staging.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
# db_name = 'CAPC_APIGATEWAY'
# env='staging'


db_connection_str = 'mysql+pymysql://{}:{}@{}/{}'.format(user,pwd,host,db_name)
db_connection = create_engine(db_connection_str)

In [26]:
# import os

# folder = "/home/trishalaswain/ClientData/SCP/01_to_16_tsv/SCP/"
# dfs = []
# for filename in os.listdir(folder):
#     if filename.startswith('SCP-') and filename.endswith("31.tsv"):
#         print(filename)
#         print(os.path.join(folder, filename))
#         dfs.append(pd.read_table(os.path.join(folder, filename),index_col=False))
# df = pd.concat(dfs)

In [27]:
# df.accountNumber.to_excel("accountNumber.xlsx",index=False)

In [28]:
acc=[]
for i in open('/home/trishalaswain/ClientData/Other/Accounts.txt','r').readlines():
    acc.append(i.rstrip('\n'))

In [29]:
len(acc)

2595

In [30]:
df_all=[]

query='''
SELECT DISTINCT
    am.account_number,
    dpd.nlp_request_s3_path,
    dpd.nlp_ner_entities_result_s3_path,
    am.facility_id,
    dm.service_date,
    dpd.nlp_pcs_xml_result_s3_path,
    dpd.preprocessing_request_s3_path,
    dpd.preprocessing_result_s3_path,
    dm.orignal_path,
    dm.type_id,
    dpd.created_date,
    dpd.ocr_request_s3_path,
    dpd.ocr_result_s3_path,
    dpd.eandmcs_request_s3_path,
    dpd.eandmcs_result_s3_path,
    dpd.ruleengine_request_s3_path,
    dpd.cptcs_request_s3_path,
    dpd.cptcs_result_s3_path,
    dm.account_id,
    em.id as encounter_id,
    dc.code AS suggested_code,
    dpd.id AS DPD_ID,
    dm.path as filePath,
    ec.code AS final_code
FROM
    account_mst am
        JOIN
    document_mst dm ON dm.account_id = am.id
        JOIN
    document_processing_detail dpd ON dpd.document_id = dm.id
        INNER JOIN
    encounter_mst em ON am.id = em.account_id
        INNER JOIN
    document_code dc ON dc.document_id = dm.id
        INNER JOIN
    eandm_code ec ON ec.encounter_id = em.id
WHERE
        dc.code_category = 'EANDM'
        and is_physician_document=1
        and dm.is_active=1
        and am.account_number in ({})
        and dpd.id = (SELECT MAX(id)
        FROM document_processing_detail
        WHERE document_id = dm.id)
    order by dm.created_date desc;
'''.format(','.join(['"'+str(x)+'"' for x in acc]))



In [31]:
df_query = pd.read_sql(query, con=db_connection)

In [32]:
df_query

,account_number,nlp_request_s3_path,nlp_ner_entities_result_s3_path,facility_id,service_date,nlp_pcs_xml_result_s3_path,preprocessing_request_s3_path,preprocessing_result_s3_path,orignal_path,type_id,...,eandmcs_result_s3_path,ruleengine_request_s3_path,cptcs_request_s3_path,cptcs_result_s3_path,account_id,encounter_id,suggested_code,DPD_ID,filePath,final_code
0,89394206,ezcapc/SCP/data/2602/nlp/cdp-xml/20230623/1085...,ezcapc/SCP/data/2602/nlp/ner/20230623/10859729...,312,2023-06-23 05:00:00,ezcapc/SCP/data/2602/nlp/pcs-xml/20230623/1085...,ezcapc/SCP/data/2602/preprocessing/20230623/10...,ezcapc/SCP/data/2602/preprocessing/20230623/10...,SCP/data/2602/document/20230623/648658259.pdf,274221,...,ezcapc/SCP/data/2602/eandmcs/20230623/10159542...,ezcapc/SCP/data/2602/ruleengine/20230623/70012...,ezcapc/SCP/data/2602/cptcs/20230623/10645736_6...,ezcapc/SCP/data/2602/cptcs/20230623/10645736_6...,10257532,8490728,93010,10938079,SCP/data/2602/document/20230623/648658259_2023...,99285
1,89394206,ezcapc/SCP/data/2602/nlp/cdp-xml/20230623/1085...,ezcapc/SCP/data/2602/nlp/ner/20230623/10859729...,312,2023-06-23 05:00:00,ezcapc/SCP/data/2602/nlp/pcs-xml/20230623/1085...,ezcapc/SCP/data/2602/preprocessing/20230623/10...,ezcapc/SCP/data/2602/preprocessing/20230623/10...,SCP/data/2602/document/20230623/648658259.pdf,274221,...,ezcapc/SCP/data/2602/eandmcs/20230623/10159542...,ezcapc/SCP/data/2602/ruleengine/20230623/70012...,ezcapc/SCP/data/2602/cptcs/20230623/10645736_6...,ezcapc/SCP/data/2602/cptcs/20230623/10645736_6...,10257532,8490728,E87.5,10938079,SCP/data/2602/document/20230623/648658259_2023...,99291
2,89394206,ezcapc/SCP/data/2602/nlp/cdp-xml/20230623/1085...,ezcapc/SCP/data/2602/nlp/ner/20230623/10859729...,312,2023-06-23 05:00:00,ezcapc/SCP/data/2602/nlp/pcs-xml/20230623/1085...,ezcapc/SCP/data/2602/preprocessing/20230623/10...,ezcapc/SCP/data/2602/preprocessing/20230623/10...,SCP/data/2602/document/20230623/648658259.pdf,274221,...,ezcapc/SCP/data/2602/eandmcs/20230623/10159542...,ezcapc/SCP/data/2602/ruleengine/20230623/70012...,ezcapc/SCP/data/2602/cptcs/20230623/10645736_6...,ezcapc/SCP/data/2602/cptcs/20230623/10645736_6...,10257532,8490728,N18.6,10938079,SCP/data/2602/document/20230623/648658259_2023...,99285
3,89394206,ezcapc/SCP/data/2602/nlp/cdp-xml/20230623/1085...,ezcapc/SCP/data/2602/nlp/ner/20230623/10859729...,312,2023-06-23 05:00:00,ezcapc/SCP/data/2602/nlp/pcs-xml/20230623/1085...,ezcapc/SCP/data/2602/preprocessing/20230623/10...,ezcapc/SCP/data/2602/preprocessing/20230623/10...,SCP/data/2602/document/20230623/648658259.pdf,274221,...,ezcapc/SCP/data/2602/eandmcs/20230623/10159542...,ezcapc/SCP/data/2602/ruleengine/20230623/70012...,ezcapc/SCP/data/2602/cptcs/20230623/10645736_6...,ezcapc/SCP/data/2602/cptcs/20230623/10645736_6...,10257532,8490728,99291,10938079,SCP/data/2602/document/20230623/648658259_2023...,99285
4,89394206,ezcapc/SCP/data/2602/nlp/cdp-xml/20230623/1085...,ezcapc/SCP/data/2602/nlp/ner/20230623/10859729...,312,2023-06-23 05:00:00,ezcapc/SCP/data/2602/nlp/pcs-xml/20230623/1085...,ezcapc/SCP/data/2602/preprocessing/20230623/10...,ezcapc/SCP/data/2602/preprocessing/20230623/10...,SCP/data/2602/document/20230623/648658259.pdf,274221,...,ezcapc/SCP/data/2602/eandmcs/20230623/10159542...,ezcapc/SCP/data/2602/ruleengine/20230623/70012...,ezcapc/SCP/data/2602/cptcs/20230623/10645736_6...,ezcapc/SCP/data/2602/cptcs/20230623/10645736_6...,10257532,8490728,Z99.2,10938079,SCP/data/2602/document/20230623/648658259_2023...,99291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10757,86755383,ezcapc/SCP/data/2515/nlp/cdp-xml/20230107/6125...,ezcapc/SCP/data/2515/nlp/ner/20230107/6125248_...,304,2023-01-07 06:00:00,ezcapc/SCP/data/2515/nlp/pcs-xml/20230107/6125...,ezcapc/SCP/data/2515/preprocessing/20230107/61...,ezcapc/SCP/data/2515/preprocessing/20230107/61...,SCP/data/2515/document/20230107/622160612.pdf,274194,...,ezcapc/SCP/data/2515/eandmcs/20230107/5860948_...,ezc

In [33]:
df_query.account_number=df_query.account_number.apply(lambda x:str(x))

In [34]:
df_query=df_query.groupby(["filePath", "service_date", 'facility_id', 'type_id']).first().reset_index()

In [35]:
df_query=df_query.sort_values(['facility_id', 'type_id'])

In [36]:
df_query.columns

Index(['filePath', 'service_date', 'facility_id', 'type_id', 'account_number',
       'nlp_request_s3_path', 'nlp_ner_entities_result_s3_path',
       'nlp_pcs_xml_result_s3_path', 'preprocessing_request_s3_path',
       'preprocessing_result_s3_path', 'orignal_path', 'created_date',
       'ocr_request_s3_path', 'ocr_result_s3_path', 'eandmcs_request_s3_path',
       'eandmcs_result_s3_path', 'ruleengine_request_s3_path',
       'cptcs_request_s3_path', 'cptcs_result_s3_path', 'account_id',
       'encounter_id', 'suggested_code', 'DPD_ID', 'final_code'],
      dtype='object')

In [37]:
len(df_query)

1299

In [38]:
print(df_query.groupby('facility_id').size().to_markdown())

|   facility_id |   0 |
|--------------:|----:|
|           297 |  61 |
|           304 | 169 |
|           309 | 623 |
|           312 | 248 |
|           316 |  42 |
|           317 | 115 |
|           362 |  41 |


In [39]:
# commands

In [40]:
def parse_and_create_script(x):
    print(x.facility_id.unique())
    
    df_query=x
    cmd=[]
    fac_list=[]

    for pdf,original_text,pcs,em_req,em_res,acc,ori_path, pre_req, pre_res, acc_id,ti, nlp_req, nlp_res, cpt_req, ctp_res, rule_req, enc_id,suggested_code, final_code, dpd_id,t_id, file_path in zip(df_query.ocr_request_s3_path,df_query.ocr_result_s3_path,
                                                                                          df_query.nlp_pcs_xml_result_s3_path,
                                                             df_query.eandmcs_request_s3_path,df_query.eandmcs_result_s3_path, df_query.account_number, df_query.orignal_path,
                                                             df_query.preprocessing_request_s3_path, df_query.preprocessing_result_s3_path,
                                                             df_query.account_id, df_query.type_id, 
                                                             df_query.nlp_request_s3_path, df_query.nlp_ner_entities_result_s3_path, 
                                                             df_query.cptcs_request_s3_path, df_query.cptcs_result_s3_path,
                                                             df_query.ruleengine_request_s3_path,
                                                             df_query.encounter_id,
                                                             df_query.suggested_code, df_query.final_code,
                                                             df_query.DPD_ID,
                                                             df_query.type_id,
                                                             df_query.filePath
                                                            ):
        if pcs == None or acc==None or em_req==None:
            continue
        fac_name=file_path.split("/")[2]
        cli=file_path.split("/")[0]
        fac_list.append(fac_name)

        # eandm
        print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_{}_{}_pcs.xml".format(env, pcs, fac_name,enc_id,enc_id, acc, suggested_code, final_code))
        print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_{}_{}_eandm_request.json".format(env, em_req, fac_name, enc_id,enc_id, acc, suggested_code, final_code))
        cmd.append((pcs, '{}/{}/{}_{}_{}_{}_pcs.xml'.format(fac_name, enc_id,enc_id, acc, suggested_code, final_code)))
        cmd.append((em_req, '{}/{}/{}_{}_{}_{}_eandm_req.json'.format(fac_name, enc_id,enc_id, acc, suggested_code, final_code)))
        
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_result.json".format(env, em_res,fac_name, acc_id, acc))
        # cmd.append((em_res, '{}/{}/{}_{}_eandm_res.json'.format(fac_name,dpd_id, acc, t_id)))

        # cpt
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_pcs.xml".format(env, pcs, fac_name, enc_id, dpd_id, t_id))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_cpt_request.json".format(env, cpt_req,fac_name, enc_id, dpd_id,t_id))

        # cmd.append((pcs, '{}/{}/{}_{}_pcs.xml'.format(fac_name, dpd_id, enc_id, acc)))
        # cmd.append((cpt_req, '{}/{}/{}_{}_cpt_req.json'.format(fac_name,dpd_id, enc_id,acc)))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_result.json".format(env, ctp_res,fac_name, acc_id, acc))

        # preprocess
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_preprocess_request.txt".format(env, pre_req, fac_name, enc_id,acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_preprocess_result.txt".format(env, pre_res, fac_name,enc_id, acc))

        # original path
        # print("aws s3 cp 's3://ezdi-{}-bucket/ezcapc/{}' {}/{}/{}_{}_{}_{}.pdf".format(env,ori_path,fac_name, enc_id, t_id, acc, suggested_code, final_code))
        # cmd.append((ori_path, '{}/{}/{}_{}_{}_{}.pdf'.format(fac_name, enc_id, t_id, acc, suggested_code, final_code)))
        # print("aws s3 cp 's3://ezdi-{}-bucket/ezcapc/{}' {}/{}/{}.txt".format(env,file_path,fac_name,t_id,acc))

        # ocr
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}.txt".format(env,original_text,fac_name,acc_id,acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}.pdf".format(env,pdf,fac_name,acc_id,acc))

        # nlp
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_NLPrequest.json".format(env, nlp_req,fac_name,enc_id, acc, suggested_code))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_NLPresult.json".format(env, nlp_res,fac_name, enc_id, acc))
        # cmd.append((nlp_req, '{}/{}/{}_{}_nlp_req.json'.format(fac_name,dpd_id, acc, t_id)))
        # cmd.append((nlp_res, '{}/{}/{}_{}_nlp_res.json'.format(fac_name,dpd_id, acc, t_id)))

        # rule
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_rule_req.json".format(env, rule_req,fac_name,acc_id, acc))

        # print(pre)

    cmd.sort()
    commands=[]

    for c in cmd:
        temp=c[0].rsplit('/', 1)
        # print(temp)
        if len(commands) == 0:
            commands.append(temp[0] + ' . --recursive --exclude "*" --include "' + temp[1] +'"')
        elif temp[0] in commands[-1]:
            if 'exclude' in commands[-1]:
                commands[-1] = commands[-1] + ' --include "' + temp[1] + '"'
        else:
            commands.append(temp[0] + ' . --recursive --exclude "*" --include "' + temp[1] + '"')
        # print('-----------------------------------------')
        # print(temp[0] + ' . --recursive --exclude "*" --include "' + temp[1] + '"')
    return cmd, commands, fac_name

In [41]:
def print_downlaod_files(commands,cmd, fac):
    print(len(commands))
    time_now=datetime.now()
    time_now='{}-{}-{}-{}-{}-{}'.format(time_now.day,time_now.month,time_now.hour,time_now.minute,time_now.second,time_now.microsecond)

    file=open("/home/trishalaswain/ClientData/SCP/SCP_JULY_HALF/{}_docs_download_{}.sh".format(fac,time_now),'w')
    for f in commands:
        file.write('aws s3 cp s3://ezdi-{}-bucket/'.format(env) + re.sub('_\w+.\d{3}_', '*',f) +'\n')
        # print('===================')
        # print(f)
    # file.write('sh {}_name_change_*'.format(fac))
    
    for f in cmd:
        # print(f)
        f=(f[0].rsplit('/', 1), f[1])
        # print('mv "'+f[0][1] +'" ' +f[1])
        file.write('mkdir --parents ./' + f[1].rsplit('/',1)[0] + '/; mv "' + f[0][1] + '" '+f[1]+'\n')
    file.write('tar -cJf docs_{}_JULY.tar.xz {}\n'.format(fac, fac))
    file.write('rm -r {}'.format(fac))
    file.close()


In [42]:
# def print_filename_change(cmd, fac):
#     time_now=datetime.now()
#     time_now='{}-{}-{}-{}-{}-{}'.format(time_now.day,time_now.month,time_now.hour,time_now.minute,time_now.second,time_now.microsecond)

#     file=open("/home/devanshparmar/devansh/Download/{}_name_change_{}.sh".format(fac,time_now),'w')
#     for f in cmd:
#         # print(f)
#         f=(f[0].rsplit('/', 1), f[1])
#         # print('mv "'+f[0][1] +'" ' +f[1])
#         file.write('mkdir --parents ./' + f[1].rsplit('/',1)[0] + '/; mv "' + f[0][1] + '" '+f[1]+'\n')
#     file.write('tar -cJf docs_{}.tar.xz {}\n'.format(fac, fac))
#     file.write('rm -r {}'.format(fac))
#     file.close()

In [43]:
def do_all(x):
    # print(len(commands))
    # for cmd in commands:
    # print(commands[0], commands[-1])
    
    cmd, commands, fac = parse_and_create_script(x)
    print_downlaod_files(commands, cmd, fac)
    # print_filename_change(cmd, fac)


In [44]:
df_query.groupby('facility_id').apply(lambda x : do_all(x))

[297]
aws s3 cp 's3://ezdi-production-bucket/ezcapc/SCP/data/2491/nlp/pcs-xml/20230406/8123697_635140172_20230424191934.034_result.xml' 2491/6558223/6558223_88160781_99291_99285_pcs.xml
aws s3 cp 's3://ezdi-production-bucket/ezcapc/SCP/data/2491/eandmcs/20230406/7758250_635140172_20230424192108.008_request.json' 2491/6558223/6558223_88160781_99291_99285_eandm_request.json
aws s3 cp 's3://ezdi-production-bucket/ezcapc/SCP/data/2491/nlp/pcs-xml/20230407/8123733_635140197_20230424192152.052_result.xml' 2491/6558254/6558254_88176840_I16.9_99285_pcs.xml
aws s3 cp 's3://ezdi-production-bucket/ezcapc/SCP/data/2491/eandmcs/20230407/7758282_635140197_20230424192324.024_request.json' 2491/6558254/6558254_88176840_I16.9_99285_eandm_request.json
aws s3 cp 's3://ezdi-production-bucket/ezcapc/SCP/data/2491/nlp/pcs-xml/20230414/8105850_634485626_20230424050301.001_result.xml' 2491/6543111/6543111_88285568_99291_99285_pcs.xml
aws s3 cp 's3://ezdi-production-bucket/ezcapc/SCP/data/2491/eandmcs/20230414

""


In [45]:
df_query.columns

Index(['filePath', 'service_date', 'facility_id', 'type_id', 'account_number',
       'nlp_request_s3_path', 'nlp_ner_entities_result_s3_path',
       'nlp_pcs_xml_result_s3_path', 'preprocessing_request_s3_path',
       'preprocessing_result_s3_path', 'orignal_path', 'created_date',
       'ocr_request_s3_path', 'ocr_result_s3_path', 'eandmcs_request_s3_path',
       'eandmcs_result_s3_path', 'ruleengine_request_s3_path',
       'cptcs_request_s3_path', 'cptcs_result_s3_path', 'account_id',
       'encounter_id', 'suggested_code', 'DPD_ID', 'final_code'],
      dtype='object')

In [46]:
# df_query[df_query.account_number.str.contains('0996')]

In [47]:
# f=open("/home/devanshparmar/devansh/Download/docs_chs.sh".format(cli,fac_name),'a')
# for d in cmd:
#     f.write(d+'\n')
    
# f.write('tar -cJf docs.tar.xz *\n')
# f.write('echo done\n')
# f.close()

In [48]:

# for f_id in type_id:
# query ='''
# SELECT DISTINCT
#     am.account_number,
#     dpd.nlp_request_s3_path,
#     dpd.nlp_ner_entities_result_s3_path,
#     am.facility_id,
#     dm.service_date,
#     dpd.nlp_pcs_xml_result_s3_path,
#     dpd.preprocessing_request_s3_path,
#     dpd.preprocessing_result_s3_path,
#     dm.orignal_path,
#     dm.type_id,
#     dpd.created_date,
#     dpd.ocr_request_s3_path,
#     dpd.ocr_result_s3_path,
#     dpd.eandmcs_request_s3_path,
#     dpd.eandmcs_result_s3_path,
#     dpd.ruleengine_request_s3_path,
#     dpd.cptcs_request_s3_path,
#     dpd.cptcs_result_s3_path,
#     dm.account_id,
#     dpd.id AS DPD_ID,
#     dm.path AS filePath,
#     dm.id as document_id,
#     em.id as encounter_id
# FROM
#     account_mst am
#         INNER JOIN
#     document_mst dm ON dm.account_id = am.id
#         INNER JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     encounter_mst em ON dm.account_id = em.account_id
#         AND dm.service_date = em.service_date
#         AND dm.physician_id = em.physician_id
#         AND em.is_active = 1
# WHERE
#         is_physician_document = 1
#         AND dm.is_active = 1
#         and account_number in ({})
#         and dpd.id = (select MAX(id)FROM document_processing_detail
#           WHERE document_id = dm.id)

# ORDER BY dm.created_date DESC;
# '''.format(','.join(['"'+str(x)+'"' for x in acc]))
# print(query)

# for f in fac:

# query='''
# SELECT DISTINCT
#     am.account_number,
#     dpd.nlp_request_s3_path,
#     dpd.nlp_ner_entities_result_s3_path,
#     am.facility_id,
#     dm.service_date,
#     dpd.nlp_pcs_xml_result_s3_path,
#     dpd.preprocessing_request_s3_path,
#     dpd.preprocessing_result_s3_path,
#     dm.orignal_path,
#     dm.type_id,
#     dpd.created_date,
#     dpd.ocr_request_s3_path,
#     dpd.ocr_result_s3_path,
#     dpd.eandmcs_request_s3_path,
#     dpd.eandmcs_result_s3_path,
#     dpd.ruleengine_request_s3_path,
#     dpd.cptcs_request_s3_path,
#     dpd.cptcs_result_s3_path,
#     dm.account_id,
#     em.id as encounter_id,
#     dc.code AS suggested_code,
#     dpd.id AS DPD_ID,
#     dm.path as filePath,
#     ec.code AS final_code
# FROM
#     account_mst am
#         JOIN
#     document_mst dm ON dm.account_id = am.id
#         JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     encounter_mst em ON am.id = em.account_id
#         left JOIN
#     document_code dc ON dc.document_id = dm.id
#         left JOIN
#     eandm_code ec ON ec.encounter_id = em.id
# WHERE
#     dc.code_category = 'EANDM'
#         and is_physician_document=1
#         and dm.is_active=1
#         and account_number in ({})
#         and dpd.id = (SELECT MAX(id)
#         FROM document_processing_detail
#         WHERE document_id = dm.id)
#     order by dm.created_date desc;
# '''.format(','.join(str(x) for x in acc))

# query = '''SELECT DISTINCT
#     am.account_number,
#     dpd.nlp_request_s3_path,
#     dpd.nlp_ner_entities_result_s3_path,
#     am.facility_id,
#     dm.service_date,
#     dpd.nlp_pcs_xml_result_s3_path,
#     dpd.preprocessing_request_s3_path,
#     dpd.preprocessing_result_s3_path,
#     dm.orignal_path,
#     dm.type_id,
#     dpd.created_date,
#     dpd.ocr_request_s3_path,
#     dpd.ocr_result_s3_path,
#     dpd.eandmcs_request_s3_path,
#     dpd.eandmcs_result_s3_path,
#     dpd.ruleengine_request_s3_path,
#     dpd.cptcs_request_s3_path,
#     dpd.cptcs_result_s3_path,
#     dm.account_id,
#     em.id AS encounter_id,
#     dc.code AS suggested_code,
#     dpd.id AS DPD_ID,
#     dm.path AS filePath,
#     ec.code AS final_code
# FROM
#     account_mst am
#         JOIN
#     document_mst dm ON dm.account_id = am.id
#         JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     encounter_mst em ON am.id = em.account_id
#         LEFT JOIN
#     document_code dc ON dc.document_id = dm.id
#         LEFT JOIN
#     eandm_code ec ON ec.encounter_id = em.id
# WHERE
#         dc.code_category = 'EANDM'
#         is_physician_document = 1
#         AND am.account_number in ({})
#         AND dm.is_active = 1
#         AND am.is_active = 1
# ORDER BY dpd.created_date DESC;'''.format(','.join(str(x) for x in acc))

           
# query = '''SELECT DISTINCT
#     am.account_number,
#     dpd.nlp_request_s3_path,
#     dpd.nlp_ner_entities_result_s3_path,
#     am.facility_id,
#     dm.service_date,
#     dpd.nlp_pcs_xml_result_s3_path,
#     dpd.preprocessing_request_s3_path,
#     dpd.preprocessing_result_s3_path,
#     dm.orignal_path,
#     dm.type_id,
#     dpd.created_date,
#     dpd.ocr_request_s3_path,
#     dpd.ocr_result_s3_path,
#     dpd.eandmcs_request_s3_path,
#     dpd.eandmcs_result_s3_path,
#     dpd.ruleengine_request_s3_path,
#     dpd.cptcs_request_s3_path,
#     dpd.cptcs_result_s3_path,
#     dm.account_id,
#     em.id AS encounter_id,
#     dc.code AS suggested_code,
#     dpd.id AS DPD_ID,
#     dm.path AS filePath,
#     ec.code AS final_code
# FROM
#     account_mst am
#         JOIN
#     document_mst dm ON dm.account_id = am.id
#         JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     encounter_mst em ON am.id = em.account_id
#         LEFT JOIN
#     document_code dc ON dc.document_id = dm.id
#         LEFT JOIN
#     eandm_code ec ON ec.encounter_id = em.id
# WHERE
#     am.facility_id = 38
#         AND is_physician_document = 1
#         AND dm.service_date >= '2023-05-01'
#         AND dm.is_active = 1
#         AND am.is_active = 1
# ORDER BY dpd.created_date DESC
# '''